In [ ]:
#assign a score for each course, based on how well the course prepare the student for the job direction. 
import csv
#tagging a file path for the .csv file.
csv_file_path = "\location"
#a dictionary to store course and course description. 
course_info = {}

with open(csv_file_path, mode='r') as file: 
    csv_reader = csv.reader(file)
    for row in csv_reader: 
        course = row[0]
        description = row[6]
        course_info[course] = description
        
#necessary skills for the job direction the student is interested, provided by Isha. 
languages = []
libraries = []
skillsets = []
    
def rate_course_on_description(course_description: str) -> int:
    language_count = 0
    for language in languages:
        if language.lower() in course_description.lower():
            language_count += 1
    library_count = 0
    for library in libraries: 
        if library.lower() in course_description.lower():
            library_count += 1
    skillset_count = 0
    for skillset in skillsets: 
        if skillset.lower() in course_description.lower(): 
            skillset_count += 1
    language_score = language_count // len(languages)
    library_score = library_count // len(libraries)
    skillset_score = skillset_count // len(skillsets)
    return (0.2 * language_score + 0.3 * library_score + 0.5 * skillset_score)

#dictionary, the score we rate each course with, based on whether the course description match well with student's job direction. 
course_jobscore = {}
for course in course_info:
    course_jobscore[course] = rate_course_on_description(course_info[course])

In [1]:
#ranking courses based on whether they need prerequisite or not, and whether they serve as a prerequisite for others. 
#key, course code. value: code for the prerequisite for that course. 
course_prereq = {}
# key, course code. Value: the score earned by the course with respect to prerequisite aspect. 
course_prereqscore = {}
with open(csv_file_path, mode='r') as file: 
    csv_reader = csv.reader(file)
    
    for row in csv_reader: 
        course = row[0]
        prerequisite = row[3]
        course_prereq[course] = prerequisite
        
#each course at most obtain a score of 1, with respect to the prerequisite aspect. 
#for courses not demanding prerequisites, give a base score of 0.4, those demanding prerequisites a base score of 0.1
for course, prerequisite in course_prereq: 
    if prerequisite == None: 
        course_prereqscore[course] = 0.4
    else:
        course_prereqscore[course] = 0.1
        
#dictionary, how many times does each course serve as a prerequisite for other courses
course_itselfasprereq = {}
for course, prerequisite in course_prereq: 
    course_itselfasprereq[course] = 0

for course, prerequisite in course_prereq: 
        course_itselfasprereq[prerequisite] += 1

#now, we have a dictionary of key be each course, and the value be the amount of time it serves as a prerequisite. 
denominator = max(course_itselfasprereq, key=course_itselfasprereq.get)
for course in course_prereqscore:
    course_prereqscore[course] += ((course_itselfasprereq[course] / denominator) - 0.4)

In [ ]:
course_ultimatescore = {}
for course in course_jobscore: 
    course_ultimatescore[course] = course_jobscore[course] + course_prereqscore[course]
    
#sort course based on score, from highest to lowest. 
course_ultimatescore_sorted = dict(sorted(course_ultimatescore.items(), key=lambda item: item[1], reverse=True))
